# Assignment no.2: 
Implementation of information retrieval system (TREC Collection)

## 💬 Commentary
This solution was supposed to implement similar solution to the one from first assignment 
(https://github.com/adamhospodka/Information_Retrieval_System/blob/main/Implementation_of_information_retrieval_system.ipynb) 
adding the ML capabilities.

However due to time/performance issues I had to downgrade from the additional ML process. It only relies on TF-IDF model.

Unfortunatelly, even this simple solution still scores *0.00 MAP*. I give up. Again.



Steps:

1. List every word from every document using ```multi_buildPairs()``` function
2. Remove duplicates using the ```uniq()``` function
3. Based on these pairs create inverted index using the ```buildInvertedIndex()``` function
4. Use knowledge of inverted index to build frequency index (inv. index with word frequencies) using the ```multi_buildFrequencyIndex()``` function
5. Build index that contains length of each document using the ```buildDocumentsLengthIndex()``` function
6. Prepare Pandas DataFrame with document instances ready to be ranked using the ```buildRankingDf()``` function
7. Initialize the search with ```IRSystem.search()```
8. Clean the given query with the ```cleanQuery()``` function
9. Rank the relevant documents with the ```rank()``` function

## 📚 Imports

In [ ]:
! pip install git+https://gitlab.fi.muni.cz/xstefan3/pv211-utils.git@master | grep '^Successfully'
! pip install pandas
! pip install numpy
! pip install nltk

In [2]:
import math
import time
import pickle
import json
import numpy as np
import pandas as pd

import nltk
from nltk.corpus import stopwords

from tqdm import tqdm
from collections import OrderedDict
from itertools import chain

from pv211_utils.trec.loader import load_documents
from pv211_utils.trec.entities import TrecQueryBase
from pv211_utils.trec.loader import load_queries
from pv211_utils.trec.entities import TrecDocumentBase
from pv211_utils.trec.loader import load_judgements
from pv211_utils.trec.leaderboard import TrecLeaderboard
from pv211_utils.trec.eval import TrecEvaluation
from pv211_utils.trec.irsystem import TrecIRSystemBase

from multiprocessing import Pool
from typing import Iterable


In [8]:
from nltk.corpus import stopwords
stopwords = stopwords.words('english')

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')

## 🏗 Classes & Instances loading

#### Documents/Corpus

In [9]:
class Document(TrecDocumentBase):
    """
    A preprocessed TREC collection document
    """

    def __init__(self, document_id: str, body: str):        
        super().__init__(document_id, body)

In [10]:
documents = load_documents(Document, cache_download='/var/tmp/pv211/trec_documents.json.gz')

Computing MD5: /var/tmp/pv211/trec_documents.json.gz
MD5 matches: /var/tmp/pv211/trec_documents.json.gz


In [11]:
CORPUS = documents

#### Queries

In [12]:
class Query(TrecQueryBase):
    """
    A preprocessed TR`EC collection query
    """    

    def __init__(self, query_id: int, title: str, body: str, narrative: str):

        super().__init__(query_id, title, body, narrative)

In [13]:
train_queries = load_queries(Query, 'train')
validation_queries = load_queries(Query, 'validation')

bigger_train_queries = OrderedDict(chain(train_queries.items(), validation_queries.items()))

#### Judgements

In [14]:
train_judgements = load_judgements(train_queries, documents, 'train')
validation_judgements = load_judgements(validation_queries, documents, 'validation')

bigger_train_judgements = train_judgements | validation_judgements

---

## ⚙️ Preprocessing

#### Tokenizing bodies

In [2]:
def tokenize(document):
    """
    Creates a dictionary with document ids (keys) and list of stemmed, tokenized terms (values).
    Constructed to be compatible with the pool.map() function.
    """
    
    doc_id = document.document_id
    body = document.body
    body = nltk.word_tokenize(body)
    body = [stemmer.stem(token) for token in body]
    
    return {doc_id: body}

In [ ]:
# Parallel processing
with Pool(None) as pool:
    tokenized_bodies = pool.map(tokenize, tqdm(CORPUS.values()) )

In [ ]:
# Reforming
tokenized_bodies_unzipped = {}
for document in tokenized_bodies:
    key = list(document.keys())[0]
    tokenized_bodies_unzipped[key] = document[key]

##### Save and load pickle

In [ ]:
# Save
with open('/data/xhospodk/tokenized_bodies_unzipped.pkl', 'wb') as f:
    pickle.dump(tokenized_bodies_unzipped, f)

In [15]:
# Load
with open('/data/xhospodk/tokenized_bodies_unzipped.pkl', 'rb') as f:
    tokenized_bodies_unzipped = pickle.load(f)
print("")

#### Build Pairs

In [24]:
def multi_buildPairs(document):
    """
    Crawls every document instance and creates a tuples -> (token, document id). 
    It only does it so when token obeys certain conditions.
    """
    
    pairs = []
    punc = '''!(),---[]{};::'"\ <>./?@#$%^&*_␣~\n\n'''
       
    tokens = tokenized_bodies_unzipped[document.document_id]

    token = ""
    doc_id = ""

    for token in tokens:
        if len(token) > 2:
            if token not in stopwords:
                for letter in token:
                    if letter in punc:
                        token = token.replace(letter, "")
                            
                doc_id = document.document_id
                pairs.append((token, doc_id))
    
    return pairs

In [25]:
# Parallel processing
with Pool() as pool:
    multi_pairs = pool.map(multi_buildPairs, tqdm(CORPUS.values() ))

100%|██████████| 527890/527890 [01:47<00:00, 4927.95it/s] 


In [26]:
# Reforming
multi_pairs_unzipped = []
for array_of_tuples in tqdm(multi_pairs):
    for tuple in array_of_tuples:
        multi_pairs_unzipped.append(tuple)

100%|██████████| 527890/527890 [00:38<00:00, 13772.10it/s]


#### Uniq

In [27]:
def uniq(sorted_list):
    """
    Reduces the list of pairs only to unique tuples (a term can be only once in a document)
    """
    
    if len(sorted_list) <= 1:
        return sorted_list

    uniq_list = sorted_list[:1]
    previous_value = sorted_list[0]

    for value in sorted_list[1:]:
        if value != previous_value:
            uniq_list.append(value)
            previous_value = value
                
    return uniq_list

In [28]:
uniq_pairs = tqdm(uniq(sorted(multi_pairs_unzipped, key=lambda x: (x[0].lower(), x[1]))))

  0%|          | 0/84576249 [00:00<?, ?it/s]

#### Inverted Index

In [35]:
def buildInvertedIndex(uniq_pairs):
    """ 
    Creates inverted index using the list of unique pairs (token, document id).
    Returns inverted index (dictionary)
    """

    inverted_index = {}

    for term, document_id in tqdm(uniq_pairs):
        if term not in inverted_index:
            inverted_index[term] = []

        inverted_index[term].append(document_id)
    
    return inverted_index

In [36]:
inverted_index = buildInvertedIndex(uniq_pairs)


100%|██████████| 84576249/84576249 [07:48<00:00, 180674.36it/s]


##### Save and load pickle

In [39]:
# Save
with open('/data/xhospodk/inverted_index_2.pkl', 'wb') as f:  # Save
    pickle.dump(inverted_index, f)

In [17]:
# Load
with open('/data/xhospodk/inverted_index_2.pkl', 'rb') as f:
    inverted_index = pickle.load(f)
    

#### Frequency Index

In [27]:
def multi_buildFrequencyIndex(key):
    """ 
    Creates frequency index (inverted index with term frequencies)
    using inverted index as heuristics for crawling.
    Returns frequency index (dictinoary).
    """
  
    local_list = {}
    relevant_documents = inverted_index[key]

    for doc_id in relevant_documents:
      body = tokenized_bodies_unzipped[doc_id]
      frequency = body.count(key)
      local_list[doc_id] = frequency
    
    insert_list = {}
    insert_list[key] = local_list

    return insert_list


In [28]:
# Parallel processing
with Pool(processes=60) as pool:
    frequency_index = pool.map(multi_buildFrequencyIndex, tqdm(inverted_index.keys()) )
    
    

100%|██████████| 1261952/1261952 [05:16<00:00, 3984.01it/s] 


In [29]:
print("done with frequencies!")

done with frequencies!


In [52]:
# Reforming
frequency_index_unzipped = {}
for document in frequency_index:
    key = list(document.keys())[0]
    frequency_index_unzipped[key] = document[key]

In [54]:
frequency_index_unzipped["car"]

{'FBIS3-10059': 1,
 'FBIS3-10134': 1,
 'FBIS3-1029': 3,
 'FBIS3-10291': 1,
 'FBIS3-1030': 8,
 'FBIS3-1031': 8,
 'FBIS3-10343': 1,
 'FBIS3-10354': 1,
 'FBIS3-10360': 1,
 'FBIS3-10361': 2,
 'FBIS3-10412': 3,
 'FBIS3-10415': 1,
 'FBIS3-10491': 1,
 'FBIS3-10538': 3,
 'FBIS3-10562': 2,
 'FBIS3-10609': 1,
 'FBIS3-1064': 1,
 'FBIS3-10644': 1,
 'FBIS3-10696': 2,
 'FBIS3-10710': 1,
 'FBIS3-10817': 1,
 'FBIS3-10897': 2,
 'FBIS3-1092': 1,
 'FBIS3-10923': 1,
 'FBIS3-10924': 2,
 'FBIS3-11011': 1,
 'FBIS3-11013': 1,
 'FBIS3-11028': 2,
 'FBIS3-11031': 1,
 'FBIS3-11072': 1,
 'FBIS3-1108': 1,
 'FBIS3-11088': 1,
 'FBIS3-11099': 1,
 'FBIS3-11108': 2,
 'FBIS3-11232': 1,
 'FBIS3-1129': 1,
 'FBIS3-1135': 1,
 'FBIS3-11459': 3,
 'FBIS3-11464': 1,
 'FBIS3-1148': 11,
 'FBIS3-11523': 1,
 'FBIS3-11568': 2,
 'FBIS3-11664': 3,
 'FBIS3-11716': 1,
 'FBIS3-11747': 1,
 'FBIS3-11770': 2,
 'FBIS3-11822': 1,
 'FBIS3-11844': 1,
 'FBIS3-11883': 3,
 'FBIS3-12010': 1,
 'FBIS3-12041': 1,
 'FBIS3-12114': 1,
 'FBIS3-12152': 1,
 

##### Save and load pickle

In [56]:
with open('/data/xhospodk/frequency_index_unzipped.pkl', 'wb') as f:  # Save
    pickle.dump(frequency_index_unzipped, f)

In [ ]:
with open('/data/xhospodk/frequency_index_unzipped.pkl', 'rb') as f:
    frequency_index_unzipped = pickle.load(f)

#### Documents Length Index

In [66]:
def buildDocumentsLengthIndex():
    """ 
    Creates index containing information about documents lengths.
    Returns documents length index (dictionary).
    """

    documents_length_index = {}

    for key in tqdm(CORPUS.keys()):
        documents_length_index[key] = len(tokenized_bodies_unzipped[key])
    
    return documents_length_index

In [67]:
documents_length_index = buildDocumentsLengthIndex()

100%|██████████| 527890/527890 [00:02<00:00, 233160.61it/s]


## 🧮 Ranking Dataframe

In [63]:
def buildRankingDf():
    """ 
    Prepares dataframe (table) with document instances loaded.
    Ranking and sorting will be happening within this structure once called.
    Returns dataframe.
    """
    
    id_as_list = [key for key, value in tqdm(CORPUS.items())]
    text_as_list = [value.body[:10] for key, value in tqdm(CORPUS.items())]

    df = pd.DataFrame({"ID":  id_as_list, "Values": text_as_list, "Rank": 0, "Matches": "" }) 
    df = df.set_index("ID")

    return df

In [64]:
ranking_df = buildRankingDf()

100%|██████████| 527890/527890 [00:02<00:00, 218084.81it/s]


In [51]:
ranking_df.shape

(527890, 3)

## 🧹 Query Cleaning

In [61]:
def cleanQuery(query):
  """ 
  Cleans passed query's body.
  Tokenization, stemming, stopwords and (words< 1) removed.
  Returns clean query body (array).
  """        

  query = query.body
  query = nltk.word_tokenize(query)
  query = [stemmer.stem(token) for token in query]
  query = [token for token in query if len(token) > 1 if token not in stopwords]

  return query

___

## 💯 Ranking Mechanism

In [134]:
def _rank(query):
    """ 
    For every token in query ranks documents +1 for every token appearing in the document
    """

    global ranking_df
    ranking_df["Rank"] = 0.0
    ranking_df["Matches"] = ""

    for term in query:
        if term in inverted_index:
            for relevant_document_id in inverted_index[term]:
                if term not in ranking_df.at[relevant_document_id, "Matches"]:
                    ranking_df.at[relevant_document_id, "Rank"] += 1
                    ranking_df.at[relevant_document_id, "Matches"] +=  " " + term


    ranking_df = ranking_df.sort_values("Rank", ascending = False)
    sorted_documents = ranking_df["Values"].tolist()

    return sorted_documents

In [70]:
def rank(query):

  global ranking_df
  ranking_df["Rank"] = 0.0
  ranking_df["Matches"] = ""


  for term in query:
    if term in frequency_index_unzipped:     
      for doc_id in frequency_index_unzipped[term]:

        # TF-IFD scoring (+ 1 point)
        tf = frequency_index_unzipped[term][str(doc_id)] / documents_length_index[str(doc_id)]
        idf = math.log(len(CORPUS) / len(frequency_index_unzipped[term]),2)
        score = tf * idf
        ranking_df.at[doc_id, "Rank"] += 1 + score


        # Adding found terms to dataframe
        ranking_df.at[doc_id, "Matches"] += " " + term



  ranking_df = ranking_df.sort_values("Rank", ascending = False)
  sorted_documents = ranking_df["Values"].tolist()

  return sorted_documents

## 🚧 Information retrieval system and evaluation

In [71]:
class IRSystem(TrecIRSystemBase):

    def __init__(self, print_matrix):
        self.print_matrix = print_matrix
        random_documents = sorted(set(document for query, document in train_judgements))
        self.random_documents = random_documents

    def search(self, query: Query) -> Iterable[Document]:
        query = cleanQuery(query)
        sorted_documents = rank(query)

        if self.print_matrix == True:
          print("Query: ",query)
          print(ranking_df.head(20))

        return(sorted_documents)

In [72]:
submit_result = False
author_name = "Hospodka, Adam"
system = IRSystem(print_matrix = True)


print('Initializing your system ...', end='', flush=True)
test_queries = load_queries(Query, 'test')
test_judgements = load_judgements(test_queries, documents, 'test')
evaluation = TrecEvaluation(system, test_judgements, TrecLeaderboard(), author_name)
print(end='\r', flush=True)
evaluation.evaluate(tqdm(test_queries.values(), desc="Querying your system, brother", leave=False), submit_result)

Querying your system, brother:   0%|          | 0/50 [00:00<?, ?it/s]

Query:  ['languag', 'cultur', 'differ', 'imped', 'integr', 'foreign', 'minor', 'germani']
                     Values      Rank  \
ID                                      
FBIS3-42766      Language:   8.030345   
FBIS3-36078      Language:   8.013373   
FBIS4-20472      Policy (Wh  8.011400   
FT941-15167      Any multin  7.078652   
LA110690-0075    The German  7.067467   
FBIS3-31659      Language:   7.064195   
FBIS3-59436      Language:   7.061007   
FBIS3-19645      Language:   7.060459   
FBIS3-20098      Language:   7.047353   
FBIS3-7981       Language:   7.040084   
FBIS4-57153      BFN [Inter  7.039182   
FBIS4-55313    BFN \n\n  [T  7.037865   
FBIS4-68774      OFFICIAL U  7.037420   
FBIS3-19685      Language:   7.033463   
FT933-4827       The intens  7.033410   
FBIS4-30880      BFN [Secon  7.032129   
FBIS3-31620      Language:   7.031881   
FBIS4-46935      Russia' Co  7.028329   
FT924-107        THE DEAD P  7.027245   
FBIS3-24338      Language:   7.027122   

       

Querying your system, brother:   2%|▏         | 1/50 [00:20<16:56, 20.74s/it]

Query:  ['happen', 'field', 'behavior', 'genet', 'studi', 'rel', 'influenc', 'genet', 'environment', 'factor', 'individu', "'s", 'behavior', 'person']
                       Values       Rank  \
ID                                         
FBIS4-20472        Policy (Wh  13.018806   
LA062590-0042      At a confe  12.121217   
LA100790-0251      Everything  12.113517   
FR940727-2-00007   National O  12.034429   
FR940104-0-00034   21 CFR Par  12.008562   
LA122489-0017      THE WORLD   11.043373   
FR941130-0-00122  Wednesday\n  11.036263   
FR940112-1-00044  Wednesday\n  11.035079   
FBIS3-41666        Federation  11.028064   
FBIS3-61193        Language:   11.021503   
FBIS3-54460        Language:   11.017301   
FBIS3-40388        OFFICIAL U  11.014499   
FBIS3-36078        Language:   11.013122   
FBIS3-60342        the Former  11.012886   
FBIS3-37947        Language:   11.009890   
FBIS3-14832        Language:   11.009069   
FBIS4-68893        CSO [Forew  11.008212   
FBIS3-24469  

Querying your system, brother:   4%|▍         | 2/50 [00:42<17:11, 21.48s/it]

...

Query:  ['signific', 'figur', 'year', 'wa', 'late', 'jordanian', 'king', 'hussein', 'peac', 'middl', 'east']
                     Values      Rank  \
ID                                      
FT944-14204     MONDAY 24\n  9.059367   
LA033189-0105    During his  9.050801   
FT943-13705    18\nMONDAY\n  9.045326   
FT941-12873      In a rare   8.256927   
FT943-14084      The Arab-I  8.247830   
FT924-8272       JUST as Jo  8.165728   
LA121390-0114    President   8.156369   
FT924-8277       JORDAN'S r  8.145585   
FT944-15625      Just how d  8.145257   
FT943-12140      It seems n  8.142227   
FT932-12056      WHEN King   8.141647   
FT943-9994       Coils of r  8.131532   
FT921-7455       Talking of  8.113049   
LA053190-0100    Arab leade  8.107756   
LA021989-0037    What a dif  8.101039   
LA082790-0013    As I write  8.094129   
LA040189-0010    King Fahd   8.093145   
LA082690-0110    All is qui  8.078329   
FT924-3613       IF NEGOTIA  8.074447   
FT933-3603       Long after  8

Your system achieved **0.00% MAP score**.

You need at least **10%** to pass. 😢

Try playing with the preprocessing of queries and documents! 💡

Set `submit_result = True` and write your name to the `author_name` variable to submit your result to [the leaderboard](https://docs.google.com/spreadsheets/d/e/2PACX-1vQ33YdFZtGH6g2bDbkD9aLozLdVVGNuP09sRh-F9d_EY9nWntOrLHSyNATFsXw4v9lw3UA3vOzl5l0s/pubhtml). 🏆

The best submissions on the leaderboard will receive *small awards during the semester*, and some *__seriously big__ awards* after the personal check at the end of the competition (2021-05-16). Please be polite, do not spoil the game for the others, and **have fun!** 😉